In [1]:
# Model referenced in homework description was build using 1.2.2
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.10.13


In [4]:
import pickle
import pandas as pd

import numpy as np

In [26]:
year = 2023
month = 3
output_file = f'predictions_{year:04d}-{month:02d}.parquet'

In [15]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [16]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [17]:
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')


In [18]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [19]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

In [20]:
np.std(y_pred)

np.float64(6.247488852238703)

In [21]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [23]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

In [24]:
df_result

,ride_id,prediction
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [28]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [29]:
!ls -lah

total 66M
drwxrwxrwx+ 4 codespace codespace 4.0K Jun 17 17:28 .
drwxrwxrwx+ 5 codespace codespace 4.0K Jun 17 16:46 ..
drwxrwxrwx+ 2 codespace codespace 4.0K Jun 17 16:49 .ipynb_checkpoints
drwxrwxrwx+ 2 codespace codespace 4.0K Jun 17 17:13 .virtual_documents
-rw-rw-rw-  1 codespace codespace  240 Jun 17 17:18 Pipfile
-rw-r--r--  1 codespace codespace  91K Jun 17 17:18 Pipfile.lock
-rw-rw-rw-  1 codespace codespace  17K Jun 17 16:52 model.bin
-rw-rw-rw-  1 codespace codespace  66M Jun 17 17:28 predictions_2023-03.parquet
-rw-rw-rw-  1 codespace codespace 8.1K Jun 17 17:27 starter.ipynb
